# Assignment 3

## Simple and Multiple Linear Regression pt2

Author: Samuel Fredric Berg

Student ID: sb224sc

Date: 2026-01-25

Course: Machine Learning 4DT905

## Conceptual

$$Y = \hat{\beta_0} + \hat{\beta_1}X_1 + \hat{\beta_2}X_2 + \hat{\beta_3}X_3 + \hat{\beta_4}X_1X_2 + \hat{\beta_5}X_1X_3$$

$$
\hat{\beta_0} = 50 (Intercept)
$$

$$
\hat{\beta_1} = 20 (GPA)
$$

$$
\hat{\beta_2} = 0.07 (IQ)
$$

$$
\hat{\beta_3} = 35 (Level)
$$

$$
\hat{\beta_4} = 0.01 (GPA \cdot IQ)
$$

$$
\hat{\beta_5} = -10 (GPA \cdot Level)
$$

$$
X_3 = 1 \text{ for College, } 0 \text{ for High School}
$$

1.


$$
Y_c = 50 + 20X_1 + 0.07X_2 + 35 + 0.01X_1X_2 - 10X_1
$$

$$
Y_h = 50 + 20X_1 + 0.07X_2 + 0.01X_1X_2
$$

$$
Y_c - Y_h = 35 - 10X_1
$$

$$
35 - 10X_1 = 0 \implies X_1 = 3.5
$$ 


Thue, when GPA > 3.5 High School graduates earn more than College graduates.

Answer: $\boxed{iii}$

2.

$$
X_1 = 4.0
$$

$$
X_2 = 110
$$

$$
X_3 = 1
$$

$$
Y = 50 + 20(4.0) + 0.07(110) + 35 + 0.01(4.0)(110) - 10(4.0)
$$

$$
Y = 137.1
$$

Answer: $\boxed{\$137,100}$

3. False. The magnitude of a coefficient does not indicate statistical importance. To determine statistical importance we need to look at the p-values associated with that coefficient, not just its absolute value. In the presented case, the units of predictor $X_2$ (IQ) are generally $\gt 100$. A small coeffeicient for the $X_2 \cdot X_1$ term might still result in a large contribution to the model and be highly statistically significant.

Answer: $\boxed{False}$

## Practical

### Imports

In [15]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

### Load data

In [16]:
df = pd.read_csv("../data/Boston.csv", index_col=0)

In [17]:
X = df[["lstat", "rm", "nox", "dis", "ptratio"]]
Y = df["medv"]
X = sm.add_constant(X)
model1 = sm.OLS(Y, X).fit()

print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:                   medv   R-squared:                       0.708
Model:                            OLS   Adj. R-squared:                  0.705
Method:                 Least Squares   F-statistic:                     242.6
Date:                Mon, 09 Feb 2026   Prob (F-statistic):          3.67e-131
Time:                        06:26:23   Log-Likelihood:                -1528.7
No. Observations:                 506   AIC:                             3069.
Df Residuals:                     500   BIC:                             3095.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         37.4992      4.613      8.129      0.0

In [18]:
df["lstat_rm"] = df["lstat"] * df["rm"]
X = df[["lstat", "rm", "lstat_rm", "nox", "dis", "ptratio"]]
X = sm.add_constant(X)
model2 = sm.OLS(Y, X).fit()

print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                   medv   R-squared:                       0.778
Model:                            OLS   Adj. R-squared:                  0.775
Method:                 Least Squares   F-statistic:                     290.8
Date:                Mon, 09 Feb 2026   Prob (F-statistic):          2.48e-159
Time:                        06:26:23   Log-Likelihood:                -1459.9
No. Observations:                 506   AIC:                             2934.
Df Residuals:                     499   BIC:                             2963.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.1518      4.880      0.646      0.5

#### Interpretation of results
By utilizing `lstat`, `rm`, `nox`, `dis` and `ptratio` columns, the model achives an R-squared value of (0.705), but by just adding the interaction between `lstat` and `rm` the R-squared value increases to (0.775). This indicates that the interaction between `lstat` and `rm` contributes a better prediction of `medv` then just using the individual predictors alone.

#### Adding non-linear term

In [19]:
df["lstat_rm_squared"] = df["lstat_rm"] ** 2
X = df[["lstat", "rm", "lstat_rm", "lstat_rm_squared", "nox", "dis", "ptratio"]]
X = sm.add_constant(X)
model3 = sm.OLS(Y, X).fit()

print(model3.summary())

                            OLS Regression Results                            
Dep. Variable:                   medv   R-squared:                       0.781
Model:                            OLS   Adj. R-squared:                  0.778
Method:                 Least Squares   F-statistic:                     253.9
Date:                Mon, 09 Feb 2026   Prob (F-statistic):          8.05e-160
Time:                        06:26:23   Log-Likelihood:                -1455.8
No. Observations:                 506   AIC:                             2928.
Df Residuals:                     498   BIC:                             2961.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               10.5522      5.499  

#### Preform ANOVA

In [20]:
ANOVA_results = sm.stats.anova_lm(model2, model3)
print(ANOVA_results)

   df_resid          ssr  df_diff     ss_diff         F    Pr(>F)
0     499.0  9500.381881      0.0         NaN       NaN       NaN
1     498.0  9348.435955      1.0  151.945925  8.094303  0.004623


#### Conclusion from ANOVA
ANOVA test indicates that the third model (with interaction and non-linear term) is significantly better then the second model (with interaction) due to the p-value being less than 0.05 (0.004) and it having a better R-squared value.

#### Add polynomial

In [21]:
for exp in range(2, 6):
    df[f"lstat_poly_{exp}"] = df["lstat"] ** exp

X = df[
    [
        "lstat",
        "rm",
        "lstat_rm",
        "lstat_poly_2",
        "lstat_poly_3",
        "lstat_poly_4",
        "lstat_poly_5",
        "nox",
        "dis",
        "ptratio",
    ]
]
X = sm.add_constant(X)
model4 = sm.OLS(Y, X).fit()

print(model4.summary())

                            OLS Regression Results                            
Dep. Variable:                   medv   R-squared:                       0.792
Model:                            OLS   Adj. R-squared:                  0.787
Method:                 Least Squares   F-statistic:                     188.0
Date:                Mon, 09 Feb 2026   Prob (F-statistic):          1.80e-161
Time:                        06:26:23   Log-Likelihood:                -1443.5
No. Observations:                 506   AIC:                             2909.
Df Residuals:                     495   BIC:                             2956.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           33.9246      7.663      4.427   

In [22]:
ANOVA_results = sm.stats.anova_lm(model2, model4)
print(ANOVA_results)

   df_resid          ssr  df_diff     ss_diff         F    Pr(>F)
0     499.0  9500.381881      0.0         NaN       NaN       NaN
1     495.0  8903.772453      4.0  596.609428  8.292038  0.000002


#### Conclusion from ANOVA
The p-value decreased significantly (0.000002) indicating that the fourth model (with interaction and polynomial term) is statistically significantly better then the third model (with interaction and non-linear term), however with high degree polynomials there is a risk of overfitting the model to the training data, which can lead to poor generalization to new data.

In [23]:
df["log_rm"] = np.log(df["rm"])

X = df[
    [
        "lstat",
        "lstat_poly_2",
        "lstat_poly_3",
        "lstat_poly_4",
        "lstat_poly_5",
        "rm",
        "log_rm",
        "nox",
        "dis",
        "ptratio",
    ]
]
X = sm.add_constant(X)
model5 = sm.OLS(Y, X).fit()

print(model5.summary())

                            OLS Regression Results                            
Dep. Variable:                   medv   R-squared:                       0.804
Model:                            OLS   Adj. R-squared:                  0.800
Method:                 Least Squares   F-statistic:                     202.6
Date:                Mon, 09 Feb 2026   Prob (F-statistic):          7.10e-168
Time:                        06:26:23   Log-Likelihood:                -1428.4
No. Observations:                 506   AIC:                             2879.
Df Residuals:                     495   BIC:                             2925.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          172.9866     13.954     12.397   

In [24]:
ANOVA_results = sm.stats.anova_lm(model2, model5)
print(ANOVA_results)

   df_resid          ssr  df_diff     ss_diff          F        Pr(>F)
0     499.0  9500.381881      0.0         NaN        NaN           NaN
1     495.0  8386.756361      4.0  1113.62552  16.431997  1.190966e-12


#### Conclusion from ANOVA
Increase in R-squared value and the very low p-value indicates that the fifth model (with interaction, higher degree polynomial term and logarithmic term) is statistically significantly better then the fourth model (with interaction and polynomial term).

#### Load data 2

In [25]:
df2 = pd.read_csv("../data/Carseats.csv", index_col=0)
print(df2.describe(), "\n")
print(df2["ShelveLoc"].value_counts(), "\n")
print(df2["Urban"].value_counts(), "\n")
print(df2["US"].value_counts())

            Sales   CompPrice      Income  Advertising  Population  \
count  400.000000  400.000000  400.000000   400.000000  400.000000   
mean     7.496325  124.975000   68.657500     6.635000  264.840000   
std      2.824115   15.334512   27.986037     6.650364  147.376436   
min      0.000000   77.000000   21.000000     0.000000   10.000000   
25%      5.390000  115.000000   42.750000     0.000000  139.000000   
50%      7.490000  125.000000   69.000000     5.000000  272.000000   
75%      9.320000  135.000000   91.000000    12.000000  398.500000   
max     16.270000  175.000000  120.000000    29.000000  509.000000   

            Price         Age   Education  
count  400.000000  400.000000  400.000000  
mean   115.795000   53.322500   13.900000  
std     23.676664   16.200297    2.620528  
min     24.000000   25.000000   10.000000  
25%    100.000000   39.750000   12.000000  
50%    117.000000   54.500000   14.000000  
75%    131.000000   66.000000   16.000000  
max    191.000000

In [26]:
X = pd.get_dummies(df2, columns=["ShelveLoc", "Urban", "US"])
for column in X.select_dtypes("bool"):
    X[column] = X[column].astype(int)

X = X.drop(columns=["Sales"])
X = sm.add_constant(X)
Y = df2["Sales"]
model = sm.OLS(Y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.873
Model:                            OLS   Adj. R-squared:                  0.870
Method:                 Least Squares   F-statistic:                     243.4
Date:                Mon, 09 Feb 2026   Prob (F-statistic):          1.60e-166
Time:                        06:26:23   Log-Likelihood:                -568.99
No. Observations:                 400   AIC:                             1162.
Df Residuals:                     388   BIC:                             1210.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                3.3853      0.253  

#### Conclusion from model summary
The model achieves an R-squared value (0.873) which indicates that $\approx$ 87.3% of the variance can be explained by the model. The prob F-statistic (1.60e-166) indicates that the model is statistically significant due to the value < 0.05.

In [27]:
X = df2.drop(columns=["Sales", "Population", "Education", "Age", "Urban", "US"])
X = pd.get_dummies(X, columns=["ShelveLoc"])

for column in X.select_dtypes("bool"):
    X[column] = X[column].astype(int)

X["Income:Advertising"] = df2["Income"] * df2["Advertising"]
X["Price:Age"] = df2["Price"] * df2["Age"]
Y = df2["Sales"]
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.870
Model:                            OLS   Adj. R-squared:                  0.868
Method:                 Least Squares   F-statistic:                     328.2
Date:                Mon, 09 Feb 2026   Prob (F-statistic):          2.90e-168
Time:                        06:26:23   Log-Likelihood:                -573.74
No. Observations:                 400   AIC:                             1165.
Df Residuals:                     391   BIC:                             1201.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  4.1957      0

#### Conclusion form model summary
Achives an R-squared value (0.870) which is worse than the previous model and prob F-statistic (2.90e-168) which indicates that the model is more statistically significant due to the value < 0.05.

#### Beat the teacher

In [28]:
X = df2.drop(columns=["Sales"])
X = pd.get_dummies(X, columns=["ShelveLoc", "US", "Urban"])

for column in X.select_dtypes("bool"):
    X[column] = X[column].astype(int)

X["Income:Advertising"] = df2["Income"] * df2["Advertising"]
Y = df2["Sales"]
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.876
Model:                            OLS   Adj. R-squared:                  0.872
Method:                 Least Squares   F-statistic:                     227.6
Date:                Mon, 09 Feb 2026   Prob (F-statistic):          5.48e-167
Time:                        06:26:23   Log-Likelihood:                -565.00
No. Observations:                 400   AIC:                             1156.
Df Residuals:                     387   BIC:                             1208.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  3.5106      0

#### Conclusion from Beat the teacher model
I was able to to get slightly better R-squared value (0.876) with a prob F-statistic (5.48e-167) which is in between the two previous models significance. I did this by not dropping any predictors and adding interaction term between `Income` and `Advertising`.